## 실습 : Deposit

### 1단계 : 컨트랙 개발

In [1]:
%%writefile src/TestDepositEvent.sol
pragma solidity ^0.5.0;

contract TestDepositEvent {
    event Deposit(
        address indexed _from,
        bytes32 indexed _id,
        uint _value
    );
    function deposit(bytes32 _id) public payable {
        emit Deposit(msg.sender, _id, msg.value);
    }
}

Writing src/TestDepositEvent.sol


### 2단계 : 컴파일

In [2]:
!solc --abi --bin --gas src/TestDepositEvent.sol


======= src/TestDepositEvent.sol:TestDepositEvent =======
Gas estimation:
construction:
   93 + 42200 = 42293
external:
   deposit(bytes32):	2009
Binary:
6080604052348015600f57600080fd5b5060d38061001e6000396000f3fe608060405260043610601c5760003560e01c8063b214faa5146021575b600080fd5b604a60048036036020811015603557600080fd5b8101908080359060200190929190505050604c565b005b803373ffffffffffffffffffffffffffffffffffffffff167f19dacbf83c5de6658e14cbf7bcae5c15eca2eedecf1c66fbca928e4d351bea0f346040518082815260200191505060405180910390a35056fea265627a7a72315820833791e3a4151485b6afeaf53c7009ed63af7549fc4666b71bf6d4a55940e73564736f6c63430005100032
Contract JSON ABI
[{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"_from","type":"address"},{"indexed":true,"internalType":"bytes32","name":"_id","type":"bytes32"},{"indexed":false,"internalType":"uint256","name":"_value","type":"uint256"}],"name":"Deposit","type":"event"},{"constant":false,"inputs":[{"internalType":"bytes32","nam

### 3단계 : 배포

In [17]:
%%writefile src/TestDepositEventDeploy.js
var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://127.0.0.1:8345"));
var _abiArray=[{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"_from","type":"address"},{"indexed":true,"internalType":"bytes32","name":"_id","type":"bytes32"},{"indexed":false,"internalType":"uint256","name":"_value","type":"uint256"}],"name":"Deposit","type":"event"},{"constant":false,"inputs":[{"internalType":"bytes32","name":"_id","type":"bytes32"}],"name":"deposit","outputs":[],"payable":true,"stateMutability":"payable","type":"function"}];
//var _abiArray=JSON.parse(_abiStr);
var _bin="0x" + "6080604052348015600f57600080fd5b5060d38061001e6000396000f3fe608060405260043610601c5760003560e01c8063b214faa5146021575b600080fd5b604a60048036036020811015603557600080fd5b8101908080359060200190929190505050604c565b005b803373ffffffffffffffffffffffffffffffffffffffff167f19dacbf83c5de6658e14cbf7bcae5c15eca2eedecf1c66fbca928e4d351bea0f346040518082815260200191505060405180910390a35056fea265627a7a72315820833791e3a4151485b6afeaf53c7009ed63af7549fc4666b71bf6d4a55940e73564736f6c63430005100032";
//unlock the account with a password provided
//web3.personal.unlockAccount(web3.eth.accounts[0],'password');
async function deploy() {
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract from " + accounts[0]);
    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: _bin})
        .send({from: accounts[0], gas: 364124, gasPrice: '1000000000'}, function(err, transactionHash) {
                if(!err) console.log("hash: " + transactionHash); 
        })
        //.then(function(newContractInstance) {
        //    console.log(newContractInstance.options.address)
        //});
    console.log("---> The contract deployed to: " + deployed.options.address)
}
deploy()

Overwriting src/TestDepositEventDeploy.js


In [13]:
!node src/TestDepositEventDeploy.js

Deploying the contract from 0xe57031CF55DEBdd411CAa043901268201A2E21F7
hash: 0x4c0f2f863ed2407ea925963ec5630000bdd79ee2451d96afca849646c75556f5
---> The contract deployed to: 0xA051F2C36E16Ec41af3208dE4Db55535cb0E5636


In [14]:
!geth --exec "eth.getTransactionReceipt('0x4c0f2f863ed2407ea925963ec5630000bdd79ee2451d96afca849646c75556f5');" attach http://127.0.0.1:8345

{
  blockHash: "0x192bb5d0116a0261c339046048b2a542ffffc392febd6321b8f815a18fae8be7",
  blockNumber: 1,
  contractAddress: "0xa051f2c36e16ec41af3208de4db55535cb0e5636",
  cumulativeGasUsed: 99029,
  from: "0xe57031cf55debdd411caa043901268201a2e21f7",
  gasUsed: 99029,
  logs: [],
  logsBloom: "0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000",
  status: "0x1",
  to: null,
  transactionHash: "0x4c0f2f863ed2407ea925963ec5630000bdd79ee2451d96afca849646c75556f5",
  transactionIndex: 0
}


### 4단계 : 사용

In [15]:
%%writefile src/TestDepositEventUse.js
var Web3=require('web3');
var web3;
if (typeof web3 !== 'undefined') {
    web3 = new Web3(web3.currentProvider);
} else {
    var web3 = new Web3(new Web3.providers.WebsocketProvider("http://127.0.0.1:8345"));
}
var _abiArray=[{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"_from","type":"address"},{"indexed":true,"internalType":"bytes32","name":"_id","type":"bytes32"},{"indexed":false,"internalType":"uint256","name":"_value","type":"uint256"}],"name":"Deposit","type":"event"},{"inputs":[{"internalType":"bytes32","name":"_id","type":"bytes32"}],"name":"deposit","outputs":[],"stateMutability":"payable","type":"function"}];
//var _abiArray=JSON.parse(_abiStr);
var _test = new web3.eth.Contract(_abiArray, '0xA051F2C36E16Ec41af3208dE4Db55535cb0E5636');
var event = _test.events.Deposit({fromBlock: 0}, function (error, result) {
    if (!error) {
        //console.log("Event fired: " + JSON.stringify(result) + "\n---> " + JSON.stringify(result.returnValues));
        console.log("Event fired: " + JSON.stringify(result.returnValues));
        //process.exit(1);
    }
});

async function doIt() {
    const accounts = await web3.eth.getAccounts();
    console.log("Account: " + accounts[0]);
    const balanceBefore = await web3.eth.getBalance(accounts[0]);
    console.log("Balance before: " + balanceBefore);
    const value = await _test.methods.deposit("0x111")
        .send({from: accounts[0], gas: 364124, gasPrice: '1000000000'})
        //.then(function(value) {console.log("---> MyFunction called " + JSON.stringify(value) +
        //                               '\n---> '+ JSON.stringify(value.events.Deposit.returnValues));});
    console.log("---> MyFunction called " + JSON.stringify(value) +
        '\n---> '+ JSON.stringify(value.events.Deposit.returnValues));
    const balanceAfter = await web3.eth.getBalance(accounts[0]);
    console.log("Balance after: " + balanceAfter);
    console.log("Balance diff: " + (balanceBefore - balanceAfter));
}

doIt()

Overwriting src/TestDepositEventUse.js


In [16]:
!node src/TestDepositEventUse.js

Account: 0xe57031CF55DEBdd411CAa043901268201A2E21F7
Balance before: 99999900971000000000
Event fired: {"0":"0xe57031CF55DEBdd411CAa043901268201A2E21F7","1":"0x1110000000000000000000000000000000000000000000000000000000000000","2":"0","_from":"0xe57031CF55DEBdd411CAa043901268201A2E21F7","_id":"0x1110000000000000000000000000000000000000000000000000000000000000","_value":"0"}
---> MyFunction called {"transactionHash":"0xc14700a7d4ea4d61ef5264f4f5eca2e08d9be5372153261af9b5a3055f1ff552","transactionIndex":0,"blockHash":"0xd6adf6c929d523ea21c155a695f1a23b3732c9b8cdc58df50f322e45f5533195","blockNumber":2,"from":"0xe57031cf55debdd411caa043901268201a2e21f7","to":"0xa051f2c36e16ec41af3208de4db55535cb0e5636","gasUsed":23225,"cumulativeGasUsed":23225,"contractAddress":null,"status":true,"logsBloom":"0x00000000000000000000000000000000000004000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000100000000000000000000000000000000000000000000000000000020000000000000